In [ ]:
import requests
import pandas as pd

def call_api():

    api_key = '15d51c5d7f0c452f60ee6405f46f39c7083ac4d1f36b204d6e64affd708773b1c774804d3a20aaa096449d90cfcd2246e2895d4a2da36d86fda36d6b22500aae'
    password = 'ffb8bad072923c0812ae7d656620ba2f4695949749942769e74eee170ab6855f6ec8e57bcda2546152cafd8ac80fe3d5090e4e7e66a2cd958c87ec4f96770f99'

    endpoint = 'https://api.returnsportal.online/v1/customer_returns'
    params = {
        'page': 1,
        'sort': 'created_at',
        'direction': 'desc',
        'per_page': '50'
    }

    headers = {
        'Authorization': f'Bearer token="{password}", api_key="{api_key}"'
    }

    response = requests.get(endpoint, params=params, headers=headers)

    if response.status_code == 200:
        data = response.json()
        return pd.DataFrame(data['customer_returns'])

    else:
        print(f"Failed to retrieve data: {response.status_code}")
        return pd.DataFrame()

def data_export():
    
    df = call_api()
    
    if not df.empty:
        df = df[df['status'] == 'registered']
    else:
        return ('Failed to retrieve data')

    df_filtered = df[['order_number','status','order_date','items']].explode('items')

    df_filtered = df_filtered.reset_index(drop=True)

    items_df = pd.json_normalize(df_filtered['items'])

    df_merged = pd.concat([df_filtered, items_df],axis=1)

    df_final = df_merged[['order_number','order_date','registered_date','sku','total_amount','return_reason.reason_internal']]

    df_final[['order_date','registered_date']] = df_final[['order_date','registered_date']].apply(pd.to_datetime)

    df_final.loc[:,'return_request'] = df_final['registered_date'].dt.date

    df_final.loc[:,'days_order_return'] = (df_final['registered_date'] - df_final['order_date']).dt.days

    df_result = df_final[['order_number','return_request','days_order_return','sku','total_amount','return_reason.reason_internal']]

    df_result.columns = ['order','return_date','days_btw_ord_rtn','sku','amnt','reason']
    
    return df_result[df_result['return_date'] == pd.Timestamp.now().date()]